# Part 1
---
Description:

Some more reports to compile:

On the "To Keep" list that Dushi sent, could I get:

1a) How many UniProt IDs on the list are currently found on ZFIN gene records? **Answer: 25,146**.

1b) How many of those ZFIN gene records are unique? **Answer: 17,991**.


# Part 2

Of the remaining not associated with the above,

2a) How many UniProt IDs have RefSeq IDs found on ZFIN gene records? **Answer: 21,898**.

2b) How many of those ZFIN gene records are unique? **Answer: 10,488**. 


## Part 2.5

As far as the “third pass”  (#3) in this request, looking at Ensembl IDs, I was thinking more along the lines of what you did over in ticket ZFIN-8376: UniProt Review of Next Release to_delete and to_load filesIN DEVELOPMENT where you used the data in NCBI to try to match UniProt IDs to ZFIN through NCBI.  This wasn’t clear from my request.

UniProt IDs are associated with RefSeqs in NCBI Gene records.  Those NCBI gene records have links back to ZFIN but we don’t have reciprocal links because we don’t match on GFIL sequence IDs.  I’m not going to ask you to do that again here because I think we’ve got that covered.  It will definitely allow us to load more UniProt IDs.

So it looks like if we load the “To Keep” IDs based on 1) UniProt IDs currently in ZFIN, 2) associations we can make through RefSeqs and 3) associations we can make through NCBI linking to us where we don’t have links to NCBI (eliminate the GFIL problem), we will not lose much relevant coverage.


### Data for Part 2.5:

Using this method, I found 62 matches.  [See tables in the data section](#data-section25) below for more details.

# Part 3
Of the remaining not associated with the above,

3a) How many UniProt IDs have Ensembl IDs found on ZFIN gene records? **Answer: 128**.

3b) How many of those ZFIN gene records are unique? **Answer: 43**. (A0A0G2L0N1 has 2 matches: olfcg13 and 14)



### Strategy

For step 3, we need some mapping between uniprot IDs and ensembl IDs.  We get that here:
https://ftp.expasy.org/databases/uniprot/current_release/knowledgebase/idmapping/by_organism/DANRE_7955_idmapping.dat.gz

We've imported that file to a table called uniprot2ensembl with 2 columns (accession, ensdarg).


# Part 4

Of the remaining not associated with the above,

4a) How many UniProt IDs have Ensembl IDs found in the notes field of "Genomic Feature" records in ZFIN?  

**Answer: There are 838 ENSDARGs in the extnote_note table (606 uniques) and those map to 328 distinct uniprot accessions.**

4b) How many unique ZFIN gene records are these allele records associated with?

**Answer: 232 markers.**

Confused on this one?  Look at the notes field on this page: ZFIN Feature: sa35159     We were unable to assign these alleles to a specific gene in ZFIN but we needed to keep the alleles and assign a gene.  We created a gene record (in this case unm_sa35159), attached the allele and made a standard note as to which Ensembl ID the gene corresponds.  We could attach a UniProt ID to the gene based on this relationship (maybe shakey and would have to discuss).


## 4a

Looks like there are 838 ENSDARGs in the extnote_note table (606 uniques).  Of those, I can match 328 to the uniprot accessions through uniprot2ensembl:

```
A0A2R8Q3S9	ENSDARG00000100186
Q5BJA5	ENSDARG00000104501
F1QQC3	ENSDARG00000044010
Q08CH3	ENSDARG00000061989
F1RCR6	ENSDARG00000061397
Q1L673	ENSDARG00000070721
...

```

## 4b
How many unique ZFIN gene records are these allele records associated with?

Using the same uniprot2ensembl table to cross reference from these ENSDARGs to uniprot accessions and cross referencing the feature IDs with markers using the feature_marker_relationship table, I found 232 markers.


# Summary

5a) How many unique ZFIN gene records are listed on the above lists, total? **Answer: 21,093**

5b) How many ZFIN gene records currently have UniProt IDs? **Answer: 23,395**

5c) How many UniProt IDs on the "To Keep" list are not matched using matching criteria 1-4? **Answer: 10,283** 

(10,221 RefSeq entries, 62 by ZFIN ID)



# Appendix

## Appendix 1 & 2

Jump to the [code section below](#appendix) for more details.



# Queries supporting these results


## Initialize Database

In [1]:
!rm -f zfin-db-slice.db
!cp inputs/zfin-db-slice.db.gz .
!gunzip -f zfin-db-slice.db.gz

In [2]:
%reload_ext sql


In [3]:
%%sql
sqlite:///zfin-db-slice.db

## Database Prep. Queries

Create a copy of to_keep for modifying

In [4]:
%%sql

-- create copy of to_keep from uniprot
create table to_keep_m as select * from to_keep;

-- Remove the version from the refseq numbers:
update to_keep_m set primary_id = substr(primary_id, 0, LENGTH(rtrim(primary_id, '0123456789'))) where database = 'RefSeq' and primary_id like '%.%';
update to_keep_m set secondary_id = substr(secondary_id, 0, LENGTH(rtrim(secondary_id, '0123456789'))) where database = 'RefSeq' and secondary_id like '%.%';
create index primary_id_key on to_keep_m (primary_id);
create index secondary_id_key on to_keep_m (secondary_id);

-- Remove the version from ensdarg in uniprot2ensembl
update uniprot2ensembl set ensdarg = substr(ensdarg, 0, LENGTH(rtrim(ensdarg, '0123456789'))) where ensdarg like '%.%';

-- Remove the version from refseq in refseq2ncbi
update refseq2ncbi set refseq = substr(refseq, 0, LENGTH(rtrim(refseq, '0123456789'))) where refseq like '%.%';


 * sqlite:///zfin-db-slice.db
Done.
61831 rows affected.
61831 rows affected.
Done.
Done.
45704 rows affected.
55485 rows affected.


[]

In [5]:
%%sql

-- Move the comma delimited dbXrefs from the ncbi_map table to its own table for ease of querying
-- See: https://stackoverflow.com/questions/51571854/how-to-split-comma-delimited-values-into-multiple-rows-using-sqlite
create table ncbi_xrefs as
WITH RECURSIVE split(id, value, rest) AS (
   SELECT GeneID, '', dbXrefs || '|' FROM ncbi_map
   UNION ALL SELECT
   id,
   substr(rest, 0, instr(rest, '|')),
   substr(rest, instr(rest, '|')+1)
   FROM split WHERE rest!=''
)
SELECT id, value
FROM split
WHERE value!='';

 * sqlite:///zfin-db-slice.db
Done.


[]

In [6]:
%%sql

-- Move the comma delimited dbXrefs from the ncbi_map table to its own table for ease of querying
-- See: https://stackoverflow.com/questions/51571854/how-to-split-comma-delimited-values-into-multiple-rows-using-sqlite
drop table if exists xrefs;
create table xrefs as
WITH RECURSIVE split(GeneID, dbXrefs, rest) AS (
   SELECT GeneID, '', dbXrefs || '|' FROM ncbi_map
   UNION ALL SELECT
   GeneID,
   substr(rest, 0, instr(rest, '|')),
   substr(rest, instr(rest, '|')+1)
   FROM split WHERE rest!=''
)
SELECT GeneID, dbXrefs, '' as org, '' as acc
FROM split
WHERE dbXrefs!='';

-- separate the organization and the accession into columns
update xrefs set acc = replace(dbXrefs, rtrim(dbXrefs, replace(dbXrefs, ':', '')), '');
update xrefs set org = substr(dbXrefs, 0, INSTR(dbXrefs, acc) - 1);


 * sqlite:///zfin-db-slice.db
Done.
Done.
90831 rows affected.
90831 rows affected.


[]

## Section 1 Queries



### 1a
---
1a) How many UniProt IDs on the list are currently found on ZFIN gene records? Answer: 25,146. 

In [7]:
%%sql 
drop table if exists "to_keep_accessions_only";
create table to_keep_accessions_only as select distinct accession from to_keep;
drop table if exists "1a";
create table "1a" as select * from to_keep_accessions_only inner join db_link on accession = dblink_acc_num;
select count(*) from "1a";

 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.
Done.
Done.


count(*)
25146


### 1b
---
How many of those ZFIN gene records are unique? Answer: 18,082.

In [8]:
%%sql  
drop table if exists "1b";  
create table "1b" as select mrkr_abbrev, mrkr_zdb_id from marker where mrkr_zdb_id in (select dblink_linked_recid from "1a");


 * sqlite:///zfin-db-slice.db
Done.
Done.


[]

In [9]:
%%sql
select count(*) from "1b";

 * sqlite:///zfin-db-slice.db
Done.


count(*)
18082


## Section 2 Queries



### Prepare DB

Delete entries from to_keep_m that have already been matched in part 1a.  Small note, instead of doing deletions, I prefer to create a view of to_keep_m that mimics what that table would look like if we did a deletion. That way we can run each step independently.


In [10]:
%%sql
drop view if exists to_keep_after_1a;
create view to_keep_after_1a as select * from to_keep_m where accession not in (select accession from "1a");

 * sqlite:///zfin-db-slice.db
Done.
Done.


[]

### 2a


In [11]:
%%sql

create index dblink_acc_num_key on db_link (dblink_acc_num);
create table "2a" as 
    select * from to_keep_after_1a 
    join db_link on
            (primary_id = dblink_acc_num or secondary_id = dblink_acc_num)
            and database = 'RefSeq';

 * sqlite:///zfin-db-slice.db
Done.
Done.


[]

In [12]:
%%sql
select count(distinct accession) from "2a";

 * sqlite:///zfin-db-slice.db
Done.


count(distinct accession)
21898


### 2b

How many of those ZFIN gene records are unique from 2a? Answer: 10,493

In [13]:
%%sql
select count(distinct dblink_linked_recid) from "2a";

 * sqlite:///zfin-db-slice.db
Done.


count(distinct dblink_linked_recid)
10488


In [14]:
%%sql
create table "2b" as select mrkr_abbrev, mrkr_zdb_id from marker where mrkr_zdb_id in (select dblink_linked_recid from "2a"); 

 * sqlite:///zfin-db-slice.db
Done.


[]

In [15]:
%%sql
-- mimic this: delete from to_keep where accession in (select accession from "2a");
drop view if exists to_keep_after_2a;
create view to_keep_after_2a as select * from to_keep_after_1a where accession not in (select accession from "2a");

 * sqlite:///zfin-db-slice.db
Done.
Done.


[]

## Section 2.5 Queries
<a name="data-section25"></a>

In [49]:
%%sql

drop table if exists "2.5a";
create table "2.5a" as 
select accession, refseq, ncbi, dblink_linked_recid, recattrib_source_zdb_id from to_keep_after_2a k join refseq2ncbi r on k.primary_id = r.refseq join db_link d on d.dblink_acc_num = r.ncbi left join record_attribution ra on ra.recattrib_data_zdb_id = d.dblink_zdb_id;


select count(*) from "2.5a";



 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


count(*)
62


In [17]:
%%sql

-- mimic this: delete from to_keep where accession in (select accession from "2a");
drop view if exists "to_keep_after_2.5a";
create view "to_keep_after_2.5a" as select * from to_keep_after_2a where accession not in (select accession from "2.5a");

select count(*) from "to_keep_after_2.5a";

 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


count(*)
10259


In [18]:
%%sql
select * from "2.5a";

 * sqlite:///zfin-db-slice.db
Done.


accession,refseq,ncbi,dblink_linked_recid
A0A8M6Z2V1,XP_017213386,101886280,ZDB-GENE-201021-1
A0A8M6Z2V1,XP_021334760,101886280,ZDB-GENE-201021-1
A0A8M9PI15,XP_021324394,100537633,ZDB-GENE-201110-1
A0A8M9PTJ7,XP_021324395,100537633,ZDB-GENE-201110-1
A0A8M2B308,XP_005157000,101883994,ZDB-GENE-201111-2
A0A2R8QN98,XP_005157000,101883994,ZDB-GENE-201111-2
A0A8M1RHE9,XP_002663314,100333146,ZDB-GENE-201105-1
E7FBD9,XP_001922909,566378,ZDB-GENE-201110-2
A0A8M6YZB7,XP_017210347,566378,ZDB-GENE-201110-2
A0A8M6Z7P2,XP_017212877,100334628,ZDB-GENE-140106-268


## Section 3 Queries



In [19]:
%%sql
drop table if exists "3a";
create table "3a" as
select * from to_keep_after_2a tkm
left join uniprot2ensembl e on tkm.accession = e.accession
left join db_link on ensdarg = dblink_acc_num
where dblink_linked_recid is not null;
select * from "3a" limit 3;

 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


accession,entry_type,database,primary_id,secondary_id,accession:1,db,ensdarg,dblink_linked_recid,dblink_acc_num,dblink_info,dblink_zdb_id,dblink_acc_num_display,dblink_length,dblink_fdbcont_zdb_id
A3KQ92,TrEMBL,RefSeq,NP_001082930,NM_001089461,A3KQ92,Ensembl,ENSDARG00000042350,ZDB-GENE-060503-166,ENSDARG00000042350,uncurrated 2019-06-12 13:43:08.78665-07,ZDB-DBLINK-190612-4636,ENSDARG00000042350,0,ZDB-FDBCONT-061018-1
A3KQ92,TrEMBL,RefSeq,NP_001082930,NM_001089461,A3KQ92,Ensembl,ENSDARG00000042350,ZDB-GENE-060503-166,ENSDARG00000042350,,ZDB-DBLINK-200123-19158,ENSDARG00000042350,,ZDB-FDBCONT-200123-1
A3KQ92,TrEMBL,ZFIN,ZDB-GENE-060503-166,syt1b,A3KQ92,Ensembl,ENSDARG00000042350,ZDB-GENE-060503-166,ENSDARG00000042350,uncurrated 2019-06-12 13:43:08.78665-07,ZDB-DBLINK-190612-4636,ENSDARG00000042350,0,ZDB-FDBCONT-061018-1


In [20]:
%%sql
select count(*) from "3a";

 * sqlite:///zfin-db-slice.db
Done.


count(*)
128


In [21]:
%%sql
select distinct accession from "3a";

 * sqlite:///zfin-db-slice.db
Done.


accession
A3KQ92
A0A0G2KV16
F1Q6N2
Q8AW20
A0A0G2KC95
Q5TKT2
F6NGW2
E9QCW0
E7FCU2
C4XVB5


In [22]:
%%sql
drop table if exists "3b";
create table "3b" as
select distinct accession, mrkr_zdb_id, mrkr_abbrev from marker join "3a" on mrkr_zdb_id = dblink_linked_recid ;
select * from "3b" order by accession;

 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


accession,mrkr_zdb_id,mrkr_abbrev
A0A0G2KC95,ZDB-GENE-161017-91,si:cabz01069016.2
A0A0G2KM47,ZDB-GENE-090303-2,cxcl11.4
A0A0G2KM96,ZDB-GENE-130726-2,kcnj10b
A0A0G2KV16,ZDB-GENE-200114-3,ifit17
A0A0G2KZI5,ZDB-GENE-161017-26,si:dkey-240n22.9
A0A0G2L0N1,ZDB-GENE-200318-4,olfcg13
A0A0G2L0N1,ZDB-GENE-200723-1,olfcg14
A0A0G2L8W6,ZDB-GENE-161017-35,si:dkey-172k15.10
A0A1L1QZG2,ZDB-GENE-161017-3,si:dkey-172k15.18
A0A1L1QZI0,ZDB-GENE-161017-132,si:cabz01021435.4


In [23]:
%%sql
select count(*) from "3b";

 * sqlite:///zfin-db-slice.db
Done.


count(*)
43


In [24]:
%%sql
--mimic: delete from to_keep where accession in (select accession from "3a");
drop view if exists to_keep_after_3a;
create view to_keep_after_3a as select * from "to_keep_after_2.5a" where accession not in (select accession from "3a");

select count(*) from to_keep_after_3a;

 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


count(*)
10193


## Section 4 Queries

### 4a

In [25]:
%%sql
select * from extnote_note where extnote_note like '%ensdarg%' limit 1;

 * sqlite:///zfin-db-slice.db
Done.


extnote_zdb_id,extnote_data_zdb_id,extnote_note,extnote_note_type,extnote_source_zdb_id,extnote_tag
ZDB-EXTNOTE-230130-4,ZDB-ALT-210617-1,"Exon 2 of ENSDARG00000032199 (Chromosome 14: 31,176,952-31,465,905: insertion of GC at 244 bp of transcript (ENSDART00000173108.2, coding sequence only), following A (255 bp) replaced by a C",feature,ZDB-PUB-230103-8,feature


In [26]:
%%sql
--create tempnotes table with the ENSDARG isolated to its own field
create table tempnotes as select SUBSTR(extnote_note, INSTR(extnote_note, 'ENSDARG'), 18) AS ensdarg, * from extnote_note;

 * sqlite:///zfin-db-slice.db
Done.


[]

In [27]:
%%sql
select count(ensdarg) from tempnotes;

 * sqlite:///zfin-db-slice.db
Done.


count(ensdarg)
840


In [28]:
%%sql
drop table if exists "4a";
create table "4a" as
select n.ensdarg, u2e.accession from tempnotes n left join uniprot2ensembl u2e on n.ensdarg = u2e.ensdarg;
select count(distinct accession) from "4a";

 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


count(distinct accession)
329


In [29]:
%%sql
select * from "4a" limit 2;

 * sqlite:///zfin-db-slice.db
Done.


ensdarg,accession
ENSDARG00000032199,Q6V9Y8
ENSDARG00000032199,Q6V9Y8


### 4b



In [30]:
%%sql

-- create supporting table (feature_marker_ensdarg) of joins from feature_marker_relationship, notes(with ensdarg), uniprot2ensembl, marker
drop table if exists "feature_marker_ensdarg";
create table "feature_marker_ensdarg" as
select fmrel_ftr_zdb_id, fmrel_type, fmrel_mrkr_zdb_id as marker_id, m.mrkr_abbrev as gene_abbrev, tempnotes.ensdarg, extnote_note 
from feature_marker_relationship 
  join tempnotes on extnote_data_zdb_id = fmrel_ftr_zdb_id
  left join uniprot2ensembl u2e on tempnotes.ensdarg = u2e.ensdarg
  left join marker m on  fmrel_mrkr_zdb_id = mrkr_zdb_id;
select * from feature_marker_ensdarg limit 5;



 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


fmrel_ftr_zdb_id,fmrel_type,marker_id,gene_abbrev,ensdarg,extnote_note
ZDB-ALT-070426-3,is allele of,ZDB-GENE-050114-2,lmx1bb,ENSDARG00000068365,"Mutation in lmx1b (ENSDARG00000068365) that introduces a premature termination codon at 34,129,111 (T>A) on chromosome 8. Overall ear size reduced, abnormally formed semicircular canals, and slightly reduced otoliths."
ZDB-ALT-070426-3,is allele of,ZDB-GENE-050114-2,lmx1bb,ENSDARG00000068365,"Mutation in lmx1b (ENSDARG00000068365) that introduces a premature termination codon at 34,129,111 (T>A) on chromosome 8. Overall ear size reduced, abnormally formed semicircular canals, and slightly reduced otoliths."
ZDB-ALT-070427-1,is allele of,ZDB-GENE-011128-4,jag1b,ENSDARG00000013168,"G>A mutation at position 35,593,144 on chromosome 13 in jag1b (ENSDARG00000013168) disrupting a splice donor site resulting in a frameshift. Antero-posterior axis is shortened in the ear."
ZDB-ALT-070427-1,is allele of,ZDB-GENE-011128-4,jag1b,ENSDARG00000013168,"G>A mutation at position 35,593,144 on chromosome 13 in jag1b (ENSDARG00000013168) disrupting a splice donor site resulting in a frameshift. Antero-posterior axis is shortened in the ear."
ZDB-ALT-090714-8,contains innocuous sequence feature,ZDB-TGCONSTRCT-090713-1,Tg(ClzpAU14I),ENSDARG00000071738,"Family pa008 22; 1,567,618 Plus strand, Target gene- ENSDARG00000071738 180 bp upstream XM_001922442"


In [31]:
%%sql
--match ensdarg to uniprot accession
drop table if exists "4b";
create table "4b" as 
select accession, fme.* from feature_marker_ensdarg fme join uniprot2ensembl u2e on fme.ensdarg = u2e.ensdarg;
select * from "4b" limit 2;


 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


accession,fmrel_ftr_zdb_id,fmrel_type,marker_id,gene_abbrev,ensdarg,extnote_note
Q4L1M6,ZDB-ALT-070426-3,is allele of,ZDB-GENE-050114-2,lmx1bb,ENSDARG00000068365,"Mutation in lmx1b (ENSDARG00000068365) that introduces a premature termination codon at 34,129,111 (T>A) on chromosome 8. Overall ear size reduced, abnormally formed semicircular canals, and slightly reduced otoliths."
Q4L1M6,ZDB-ALT-070426-3,is allele of,ZDB-GENE-050114-2,lmx1bb,ENSDARG00000068365,"Mutation in lmx1b (ENSDARG00000068365) that introduces a premature termination codon at 34,129,111 (T>A) on chromosome 8. Overall ear size reduced, abnormally formed semicircular canals, and slightly reduced otoliths."


In [32]:
%%sql
select count(*) from (select distinct accession, marker_id from "4b");


 * sqlite:///zfin-db-slice.db
Done.


count(*)
408


In [33]:
%%sql
select count(*) from (select distinct marker_id from "4b");

 * sqlite:///zfin-db-slice.db
Done.


count(*)
232


In [34]:
%%sql
select accession, ensdarg, marker_id, gene_abbrev from "4b" limit 2;

 * sqlite:///zfin-db-slice.db
Done.


accession,ensdarg,marker_id,gene_abbrev
Q4L1M6,ENSDARG00000068365,ZDB-GENE-050114-2,lmx1bb
Q4L1M6,ENSDARG00000068365,ZDB-GENE-050114-2,lmx1bb


In [35]:
%%sql
select count( distinct accession ) from "4a" limit 5;

 * sqlite:///zfin-db-slice.db
Done.


count( distinct accession )
329


In [36]:
%%sql
--mimic: delete from to_keep where accession in (select accession from "4a");
drop view if exists to_keep_after_4a;
create view to_keep_after_4a as select * from to_keep_after_3a k where k.accession not in (select distinct accession from "4b");
select count(*) from to_keep_after_4a;

 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


count(*)
10161


## Section 5 Queries

## 5a

In [37]:
%%sql

select count (distinct dblink_linked_recid) as gene_id from (
select dblink_linked_recid from "1a"
union
select dblink_linked_recid from "2a"
union
select dblink_linked_recid from "3a"
union
select marker_id as dblink_linked_recid from "4b"
)

-- select dblink_linked_recid from "2a" limit 5;  --get gene IDs from 2a
-- select dblink_linked_recid from "3a" ;  --get gene IDs from 3a
-- select marker_id from "4b";   --get gene IDs from 4b


 * sqlite:///zfin-db-slice.db
Done.


gene_id
21093


## 5b

In [38]:
%%sql

create table "5b" as
select distinct 
	dblink_linked_recid as gene_id,
	mrkr_abbrev as gene_abbrev
FROM
	db_link dl 
	left join marker m on dl.dblink_linked_recid = m.mrkr_zdb_id
WHERE
	dblink_fdbcont_zdb_id in ( 'ZDB-FDBCONT-040412-47' ) -- uniprot is ZDB-FDBCONT-040412-47
order by dblink_linked_recid;

select count(*) from "5b";

 * sqlite:///zfin-db-slice.db
Done.
Done.


count(*)
23395


In [39]:
%%sql
select * from to_keep_after_4a where database <> 'RefSeq';


 * sqlite:///zfin-db-slice.db
Done.


accession,entry_type,database,primary_id,secondary_id
A0A8M9PFT5,TrEMBL,ZFIN,ZDB-GENE-050419-171,pgpep1l
A0A8M1RJV8,TrEMBL,ZFIN,ZDB-GENE-090311-19,lrba
A0A8M9PST5,TrEMBL,ZFIN,ZDB-GENE-050419-202,pias1b
A0A8M1NZ01,TrEMBL,ZFIN,ZDB-GENE-130530-791,prr5a
A0A8M1Q7M7,TrEMBL,ZFIN,ZDB-GENE-060503-247,emilin2a
A0A8M9PQI7,TrEMBL,ZFIN,ZDB-GENE-090313-92,snphb
A0A8M2B8K6,TrEMBL,ZFIN,ZDB-GENE-030616-129,patj
A0A8M9Q1X1,TrEMBL,ZFIN,ZDB-GENE-100921-78,cgna
A0A8M9PDG6,TrEMBL,ZFIN,ZDB-GENE-050208-336,cts12
A0A8M3AQU2,TrEMBL,ZFIN,ZDB-GENE-100922-42,hivep3b


In [40]:
%%sql
select count(*) from to_keep_after_4a where database = 'RefSeq';


 * sqlite:///zfin-db-slice.db
Done.


count(*)
10099


In [41]:
%%sql
create view extnotes as select * from tempnotes;

 * sqlite:///zfin-db-slice.db
Done.


[]

# Appendix
<a name="appendix"></a>

## Appendix 1. Error Check

Implement the logic described in comment:

To find potential problems, I was thinking it would be a good idea to 1) find all of the UniProt IDs on the “To Keep” list in ZFIN and their corresponding ZFIN gene.  Then, from the original ”To Keep” list of the UniProt IDs, 2) run all of the RefSeqs to find the corresponding ZFIN gene.  Then make sure that the ZFIN gene identified by the UniProt ID was the same ZFIN gene identified by the RefSeq.

For example, the lines in the “to Keep” file:

Q75ZI3,TrEMBL,RefSeq,NP_999853.1,NM_214688.1,
Q75ZI3,TrEMBL,ZFIN,ZDB-GENE-030131-2185,vcanb,

In the first search, Q75ZI3 is associated with vcanb and in the second search, RefSeq,NP_999853 is also associated with vcanb → OK

In the first search, Q75ZI3 is associated with vcanb and in the second search, RefSeq,NP_999853 is associated with vcana → Not OK



In [42]:
%%sql
drop table if exists to_keep_wide;
create table to_keep_wide as
select a.accession, a.entry_type, a.primary_id as zfin_id, a.secondary_id as abbrev, b.primary_id as refseq1, b.secondary_id as refseq2 from (select * from to_keep_m where database = 'ZFIN') as a 
left join (select * from to_keep_m where database = 'RefSeq') b on a.accession = b.accession;

 * sqlite:///zfin-db-slice.db
Done.
Done.


[]

In [43]:
%%sql
drop table if exists appendix1;
create table appendix1 as
SELECT
    k.*,
    d.dblink_linked_recid,
    m.mrkr_abbrev
FROM
    to_keep_wide k
    JOIN db_link d ON d.dblink_acc_num = k.refseq1
    JOIN marker m ON d.dblink_linked_recid = m.mrkr_zdb_id
WHERE zfin_id not in (select dblink_linked_recid from db_link where dblink_acc_num = k.refseq1);

-- select count(*) from appendix1;
select * from appendix1;




 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


accession,entry_type,zfin_id,abbrev,refseq1,refseq2,dblink_linked_recid,mrkr_abbrev
R4GEN1,TrEMBL,ZDB-GENE-121214-92,si:dkey-31m14.7,XP_699946,XM_694854,ZDB-GENE-100330-2,mark4b
A0A0R4IHS5,TrEMBL,ZDB-GENE-110914-181,si:dkey-262g12.7,XP_009299301,XM_009301026,ZDB-GENE-071004-60,zgc:173624
B0S673,TrEMBL,ZDB-GENE-040625-39,rpl37,XP_017207846,XM_017352357,ZDB-GENE-070912-694,coa1
A0A8M1QR37,TrEMBL,ZDB-GENE-131121-97,si:dkey-172k15.3,XP_001922501,XM_001922466,ZDB-GENE-161017-82,si:dkey-172k15.13
A0A8M1QR37,TrEMBL,ZDB-GENE-131121-97,si:dkey-172k15.3,XP_021331505,XM_021475830,ZDB-GENE-161017-82,si:dkey-172k15.13
Q568F0,TrEMBL,ZDB-GENE-050706-188,eef1a1l2,NP_001017795,NM_001017795,ZDB-GENE-050417-327,eef1a1b
Q568F0,TrEMBL,ZDB-GENE-050706-188,SO:0001217,NP_001017795,NM_001017795,ZDB-GENE-050417-327,eef1a1b
Q1MTC7,TrEMBL,ZDB-GENE-990415-150,mhc2a,NP_001004521,NM_001004521,ZDB-GENE-980526-201,mhc2daa
E9QE43,TrEMBL,ZDB-GENE-060503-165,si:dkey-28d5.5,XP_001919193,XM_001919158,ZDB-GENE-160113-18,si:ch211-63p21.3
A0A8M3APS4,TrEMBL,ZDB-GENE-110913-104,si:ch211-103e16.5,XP_009296335,XM_009298060,ZDB-GENE-081223-1,lrrc56



## Appendix 2.

Same as above, but instead of cross-checking on the refseq, just check on the uniprot accession.

In [44]:
%%sql


drop table if exists appendix2;
create table appendix2 as
SELECT
    k.*,
    d.dblink_linked_recid,
    m.mrkr_abbrev
FROM
    to_keep_wide k
    JOIN db_link d ON d.dblink_acc_num = k.accession
    JOIN marker m ON d.dblink_linked_recid = m.mrkr_zdb_id
WHERE zfin_id not in (select dblink_linked_recid from db_link where dblink_acc_num = k.accession);

-- select count(*) from appendix1;
select * from appendix2;


 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.


accession,entry_type,zfin_id,abbrev,refseq1,refseq2,dblink_linked_recid,mrkr_abbrev
R4GEN1,TrEMBL,ZDB-GENE-121214-92,si:dkey-31m14.7,XP_699946,XM_694854,ZDB-GENE-100330-2,mark4b
Q1MTC7,TrEMBL,ZDB-GENE-990415-150,mhc2a,NP_001004521,NM_001004521,ZDB-GENE-980526-201,mhc2daa
Q1MTC7,TrEMBL,ZDB-GENE-990415-150,mhc2a,NP_001092896,NM_001099426,ZDB-GENE-980526-201,mhc2daa
A0A2R8PYX0,TrEMBL,ZDB-GENE-091204-463,si:ch73-63e15.2,XP_009294614,XM_009296339,ZDB-GENE-060130-85,sbno2b
F1QVS3,TrEMBL,ZDB-GENE-041001-97,si:ch211-285c6.4,XP_005166693,XM_005166636,ZDB-GENE-080215-3,zgc:172075
A4IGE9,TrEMBL,ZDB-GENE-120214-9,pimr161,NP_001077043,NM_001083574,ZDB-GENE-070424-73,pimr166
A0A2R8QLP7,TrEMBL,ZDB-GENE-050411-48,im:7142702,XP_002667906,XM_002667860,ZDB-GENE-110201-2,bcl11bb
E9QG09,TrEMBL,ZDB-GENE-040801-94,pibf1,XP_001344916,XM_001344880,ZDB-GENE-090312-167,klf5a
A0A0R4IVW1,TrEMBL,ZDB-GENE-050208-210,si:ch73-217b7.1,XP_021325713,XM_021470038,ZDB-GENE-090313-100,map7b
F1QRI1,TrEMBL,ZDB-GENE-080227-6,ugt1a7,NP_001170810,NM_001177339,ZDB-GENE-080227-5,ugt1a6


## Appendix 3
A list of ZFIN genes that lost UniProt IDs.  All genes that currently have uniprot accessions after removing the list from 1a and 2a.

In [45]:
%%sql
drop table if exists appendix3;
drop table if exists pre_appendix3;

create table pre_appendix3 as
select distinct dblink_linked_recid, dblink_acc_num from db_link 
where dblink_fdbcont_zdb_id = 'ZDB-FDBCONT-040412-47'
and dblink_linked_recid not in (select dblink_linked_recid from "1a")
and dblink_linked_recid not in (select dblink_linked_recid from "2a");

create table appendix3 as
select dblink_linked_recid, dblink_acc_num, mrkr_abbrev
from pre_appendix3 p
left join marker m on p.dblink_linked_recid = m.mrkr_zdb_id;

select count(distinct dblink_linked_recid) from appendix3;


 * sqlite:///zfin-db-slice.db
Done.
Done.
Done.
Done.
Done.


count(distinct dblink_linked_recid)
3104


## Import

importing to sqlite from our main db requires using this query to pull data out of our zfindb:

```
select * from db_link where dblink_fdbcont_zdb_id in (
'ZDB-FDBCONT-040412-37',
'ZDB-FDBCONT-040412-36',
'ZDB-FDBCONT-131021-1',
'ZDB-FDBCONT-040412-42',
'ZDB-FDBCONT-110301-1',
'ZDB-FDBCONT-040412-1',
'ZDB-FDBCONT-061018-1',
'ZDB-FDBCONT-040412-47',
'ZDB-FDBCONT-040412-38',
'ZDB-FDBCONT-200123-1',
'ZDB-FDBCONT-040412-39'
)
```

Similarly, these tables are imported:
```
select * from marker;
select * from feature_marker_relationship;
select * from external_note where extnote_note like '%ENSDARG%'
```

Example:
```
echo "copy (select * from db_link where dblink_fdbcont_zdb_id in ( 'ZDB-FDBCONT-040412-37', 'ZDB-FDBCONT-040412-36', 'ZDB-FDBCONT-131021-1', 'ZDB-FDBCONT-040412-42', 'ZDB-FDBCONT-110301-1', 'ZDB-FDBCONT-040412-1', 'ZDB-FDBCONT-061018-1', 'ZDB-FDBCONT-040412-47', 'ZDB-FDBCONT-040412-38', 'ZDB-FDBCONT-200123-1', 'ZDB-FDBCONT-040412-39' )) to stdout with csv header" | psql -h 127.0.0.2 -U postgres zfindb > db_link.csv
```


## Playground

Here is where to put some temporary sql for playing around with the data


In [46]:
%%sql
select accession, refseq, ncbi, dblink_linked_recid, dblink_zdb_id from to_keep_after_2a k join refseq2ncbi r on k.primary_id = r.refseq join db_link d on d.dblink_acc_num = r.ncbi left join record_attribution ra on ra.recattrib_data_zdb_id = d.dblink_zdb_


 * sqlite:///zfin-db-slice.db
(sqlite3.OperationalError) no such column: d.dblink_zdb_
[SQL: select accession, refseq, ncbi, dblink_linked_recid, dblink_zdb_id from to_keep_after_2a k join refseq2ncbi r on k.primary_id = r.refseq join db_link d on d.dblink_acc_num = r.ncbi left join record_attribution ra on ra.recattrib_data_zdb_id = d.dblink_zdb_]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


# Export Excel Spreadsheet



In [50]:
import sqlite3
import pandas as pd

def main():

    tables = ['1a', '1b', '2a', '2b', '2.5a', '3a', '3b', '4a', '4b', '5b', 'extnotes', 'appendix1', 'appendix2', 'appendix3', 'to_keep_after_4a']

    # Create a Pandas Excel writer using the openpyxl engine
    writer = pd.ExcelWriter('8395 - UniProt Data.xlsx', engine='openpyxl')

    # Loop over the CSV files
    for i, table in enumerate(tables):
      # Read the CSV file
      df = get_table_rows_as_data_frame(table)

      # Write the dataframe to a sheet in the Excel file
      df.to_excel(writer, table, index=False)

    writer.close()
    print("Done")


def get_table_rows_as_data_frame(tablename):
    # Connect to the database
    conn = sqlite3.connect('zfin-db-slice.db')

    # Create a cursor
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM "' + tablename + '"')
    results = cursor.fetchall()

    column_names = [description[0] for description in cursor.description]

    # Convert the results to a Pandas DataFrame
    df = pd.DataFrame(results, columns=column_names)

    cursor.close()
    conn.close()

    return df

main()



Done
